In [59]:
import pandas as pd

In [60]:
df = pd.read_csv("spotify_millsongdata.csv")

In [61]:
df.head(5)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [62]:
df.tail(5)

,artist,song,link,text
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...
57649,Zwan,Heartsong,/z/zwan/heartsong_20148991.html,come in \r\nmake yourself at home \r\ni'm a ...


In [63]:
df.shape

(57650, 4)

In [64]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [65]:
df =df.sample(15000).drop('link', axis=1).reset_index(drop=True)


In [66]:
df.head(10)

,artist,song,text
0,Yoko Ono,Have You Seen A Horizon Lately,"Have you seen a horizon lately, \r\nIf you ha..."
1,Guns N' Roses,Estranged,When you're talkin' to yourself \r\nAnd nobod...
2,Garth Brooks,Against The Grain,Folks call me a maverick \r\nGuess I ain't to...
3,Eminem,Echo,"[Chorus:] \r\nOh, all around the World, \r\n..."
4,Bob Dylan,Freight Train Blues,I was born in Dixie in a boomer shack \r\nJus...
5,Slayer,No Remorse (I Wanna Die),It's the terror that you've gotta miss \r\nAn...
6,Rihanna,Vogue,What at you looking at? \r\n \r\nStrike a po...
7,Alice Cooper,I Love America,I love that mountain with those four big heads...
8,Dire Straits,I Think I Love You Too Much,I'm not alone sitting right there \r\nHow com...
9,NOFX,Clams Have Feelings Too,"Birds are dumb, 'cause small bird brains \r\n..."


In [67]:
df.tail(5)

,artist,song,text
14995,Nat King Cole,Stardust,And now the purple dusk of twilight time \r\n...
14996,Eddie Cochran,Sixteen Tons,Some people say a man is made outta mud \r\nA...
14997,Bob Dylan,I'd Have You Anytime,by Bob Dylan and George Harrison \r\n \r\nLe...
14998,Kris Kristofferson,Spooky Lady's Revenge,"Looky here, who's in focus now \r\nHard and c..."
14999,Pink Floyd,Seabirds,"Mighty waves come crashing down, \r\nThe spra..."


In [68]:
df['text'][0]

'Have you seen a horizon lately,  \r\nIf you have, watch it for a while.  \r\nFor you never know,  \r\nIt may not last so long.  \r\n  \r\nHave you seen an evening light lately?  \r\nIf you have, watch it for a while.  \r\nFor you never know,  \r\nIt may not be the same.  \r\n  \r\nHave you seen a snowflake lately?  \r\nIf you have, hold it in your hand.  \r\nFor you never know,  \r\nIt may melt away.  \r\n  \r\nHave you been in love lately?  \r\nIf you have, hold it in your heart.  \r\nFor you never know,  \r\nIt may be the last.\r\n\r\n'

In [69]:
df.shape

(15000, 3)

In [70]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)


In [71]:

import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

In [72]:
df['text'] = df['text'].apply(lambda x: tokenization(x))

In [73]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [74]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [75]:
similarity[0]

array([1.        , 0.11795145, 0.0096389 , ..., 0.05511748, 0.01114362,
       0.01813977], shape=(15000,))

In [76]:

df[df['song'] == 'Crying Over You']

,artist,song,text
4358,ABBA,Crying Over You,i 'm waitin ' for you babi i 'm sit all alon i...


In [77]:

def recommendation(song_df):
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])
    
    songs = []
    for m_id in distances[1:21]:
        songs.append(df.iloc[m_id[0]].song)
        
    return songs

In [78]:
recommendation('Crying Over You')

["Cryin' Drunk",
 'The Crying Game',
 "Cryin' Time Again",
 'Green Eyes',
 'Crying Time',
 'My Lovely Man',
 "Love Don't Let Me Go (Original Edit)",
 'Cryin Shame',
 'Killing Floor',
 "I Saw Gener Cryin' In His Sleep",
 'Hold Back The Tears',
 'If I Cried Every Time You Hurt Me',
 'I Want You To Want Me',
 'How Does It Feel',
 'Change My Way Of Living',
 'Blue Me',
 'Psalm 8',
 'Moment Of Forever',
 'I Want You To Want Me',
 'Back Into Blue']

In [79]:

import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))